In [1]:
import torch

import dataset
import data_reader

from torch.utils.data import DataLoader

In [2]:
EXP_NAME = "ejemplo_2_"

DATA_SET_NAME = f"data_set_x20_100%_split"

BATCH_SIZE = 1

In [3]:
def wsi_class(X):
    global net
    """
    Input: 
    - X, patches from a given WSI.
    - y, patch labels
    Outputs:
    - output, Prediction
    """
    outputs = net(X)
    y_pred = [torch.argmin(i) for i in outputs.cpu()] # 1 means positive diagnosis: (1,0) => 1
    output = y_pred.sum()/len(y_pred)

    return output


In [4]:
SPLIT_NAME = DATA_SET_NAME + f"{0}"
MODEL_NAME = EXP_NAME + f"{0}"

net = torch.load(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\results\\WSI\\models\\{MODEL_NAME}.pth") # Model loading

test_set = dataset.PatchDataset([], [], [])

X_, y_, _ , case_ids_ = data_reader.read_lmdb(f"C:/Users/Alejandro/Desktop/heterogeneous-data/data/patches/{SPLIT_NAME}")

test_set.inputs.extend(X_)
test_set.labels.extend(y_)
test_set.case_ids.extend(case_ids_)

X = test_set.inputs[:16]

X = X.type(torch.FloatTensor).to(device).permute(0, 3, 2, 1)

wsi_class(X)


Read lmdb


AttributeError: 'list' object has no attribute 'type'

In [ ]:
def test_wsi():
    global net, test_set

    # In order to obtain the correct order labels
    
    X = sorted(zip(X, ids_X))
    y = sorted(zip(y, ids_y))

    X, ids_X = zip(*X)
    y, ids_y = zip(*y)
    
    test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

    print(f"Patches for test: {len(test_set)}\n")
    

In [ ]:
# Choosing only the images from the according splits (10CV)

SPLITS = 1 # Number of iterations > n_splits

for SPLIT in range(SPLITS):
    MODEL_NAME = EXP_NAME + f"{SPLIT}"
    SPLIT_NAME = DATA_SET_NAME + f"{SPLIT}"

    test_set = dataset.PatchDataset([], [], [])

    X_, y_, _ , case_ids_ = data_reader.read_lmdb(f"C:/Users/Alejandro/Desktop/heterogeneous-data/data/patches/{SPLIT_NAME}")

    test_set.inputs.extend(X_)
    test_set.labels.extend(y_)
    test_set.case_ids.extend(case_ids_)

    print(f"Loading model {MODEL_NAME}\n")

    net = torch.load(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\results\\WSI\\models\\{MODEL_NAME}.pth") # Model loading

    test_wsi()

    del test_set


In [1]:
import torch
import torchvision.transforms as torchvision
import torch.nn as nn

from PIL import Image

from tqdm.notebook import tqdm
import glob
import os

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

## In order to fix .dll openslide bug a path for said file is provided
os.environ['PATH'] = r"C:\Users\Alejandro\Downloads\openslide-win64-20171122\openslide-win64-20171122\bin" + ";" + os.environ['PATH']  #libvips-bin-path is where you save the libvips files
import openslide
import large_image

In [2]:
def pred_data(paths, patch_size, formats, net): #  paths => paths for all data folders. dataset => train, test or val
    for path in tqdm(paths):
        #case_id = os.path.split(path)[-1]
        if not os.path.exists(path) or len(os.listdir(path)) == 0:
            print("Path does not exist")
            pass
        else:
            file_data = []
            for format in formats:
                for file in glob.glob(path + r"\*" + format):
                    if file[-51:-49] in ("01", "02", "03", "04" ,"05", "06", "07", "08", "09"): # Reading the ID diagnostic sample type 01 == Primary tumor
                        label = torch.tensor([1,0]).to("cuda:0").float()
                    else:
                        label = torch.tensor([0,1]).to("cuda:0").float()
                    file_data += pred_file(file, patch_size, label, net)
    return file_data

In [16]:
def pred_file(file, patch_size, label, net):
    """
    Classifies a WSI (file) using net classifier
    """
    device = "cuda:0"
    ts = large_image.getTileSource(file)
    
    patches = []
    labels = []
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    normalize = torchvision.transforms.Normalize(mean=mean, std=std)
    
    for tile_info in ts.tileIterator(
    scale=dict(magnification=20),
    tile_size=dict(width=patch_size, height=patch_size),
    tile_overlap=dict(x=0, y=0),
    format=large_image.tilesource.TILE_FORMAT_PIL
    ):  
        patch = tile_info['tile']

        # Changing from PIL RGBA image to RGB tensor
        patch_aux = Image.new("RGB", patch.size, (255, 255, 255))
        patch_aux.paste(patch, mask=patch.split()[3]) # 3 is the alpha channel

        patch = np.asarray(patch_aux)

        avg = patch.mean(axis=0).mean(axis=0)

        if  avg[0]< 220 and avg[1]< 220 and avg[2]< 220 and patch.shape == (patch_size, patch_size, 3): # Checking if the patch is white and its a square tile
            patch = (torch.from_numpy(patch)/255).to(device)
            patch = normalize(patch.permute(2, 1, 0))
            patches.append(patch)
            labels.append(label)
            
    patches = torch.stack(patches).view(-1, 3, patch_size, patch_size)
    labels = torch.stack(labels)

    print(len(patches))
    
    acc, loss, output = fwd_pass(patches, labels, net)
            
    return acc, loss, output

In [17]:
def fwd_pass(X, y, net, train=False):
    loss_function = loss_function = nn.BCELoss()

    outputs = net(X)

    matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs, y)]

    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    
    output = torch.sum(outputs, dim=0)
    if output[0]>output[1]:
        output = [1, 0]
    else:
        output = [0, 1]

    del outputs, X, y

    return acc, loss, output

In [18]:
data_path = r"C:\Users\Alejandro\Desktop\heterogeneous-data\data\gdc_download_20220427_144600.480657"

case_id = os.listdir(data_path)

paths = [data_path + "\\" + case for case in case_id][3:5] # All case folders paths

In [19]:
net = torch.load(r"C:\Users\Alejandro\Desktop\heterogeneous-data\results\WSI\models\ejemplo_0.pth")

In [20]:
patch_size = 512

pred_data(paths, patch_size, [".svs"], net)

  0%|          | 0/2 [00:00<?, ?it/s]

331


RuntimeError: CUDA out of memory. Tried to allocate 5.17 GiB (GPU 0; 12.00 GiB total capacity; 8.01 GiB already allocated; 2.61 GiB free; 8.03 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
label = torch.tensor([1,0]).to("cuda:0").float()
file = r"C:\Users\Alejandro\Desktop\heterogeneous-data\data\gdc_download_20220427_144600.480657\02b370ac-0168-4bd4-8178-54770b134a45\TCGA-3A-A9IL-01A-01-TS1.F811C83B-1C85-4F69-9E14-E5104E385D84.svs"
patch_size = 512

pred_file(file, patch_size, label, net)

(0.9420289855072463,
 tensor(0.1278, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>),
 [1, 0])

In [1]:
%run patch_gen_grid.py --wsi_path="C:/Users/Alejandro/Desktop/heterogeneous-data/data/gdc_download_20220427_144600.480657/feba0925-0edd-4077-a5e1-7c93d33283ae" --patch_path="C:/Users/Alejandro/Desktop/heterogeneous-data/data/patches"

C:\Users\Alejandro\Desktop\heterogeneous-data\code\WSI\patch_gen_grid.py:15: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation, binary_erosion
C:\Users\Alejandro\Desktop\heterogeneous-data\code\WSI\patch_gen_grid.py:15: DeprecationWarning: Please use `binary_erosion` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation, binary_erosion


ArgumentParser(prog='patch_gen_grid.py', usage=None, description='Generate patches from a given folder of images', formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)


100%|██████████| 2/2 [00:00<?, ?it/s]
